### <font size=5px color="#95C7AE">防止Google Colab自動中斷程式碼:</font>

<p><font size=3px > 每60分鐘自動執行程式碼刷新，解除90分鐘斷開限制.
<p><font size=3px > 使用方法：colab頁面按下F12或Ctrl+Shift+I (mac按Option+Command+I) 在console（控制台） 輸入以下程式碼並回車. </p>< b>複製以下程式碼貼在瀏覽器console！ ！ 請勿關閉瀏覽器以免失效</b>


```javascript
function ConnectButton(){
     console.log("Connect pushed");
     document.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [ ]:
#@markdown <h3>← 輸入了程式碼後運行以防止斷開</h>


import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
…
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


### <font size=3px color="#95C7AE"> main

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.sys.path.append('..')


Mounted at /content/drive


In [ ]:
!pip install scikit-multilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.4 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/mid/
from skmultilearn.model_selection import IterativeStratification
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.preprocessing.image import img_to_array, load_img
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA



folders = os.listdir('processed_data_all')

data = []
labels = []

for folder in tqdm(folders,desc='load image'):
    if not os.path.isdir(f'processed_data_all/{folder}'):
        continue
    for file in os.listdir(f'processed_data_all/{folder}'):
        image = cv2.imread(f'processed_data_all/{folder}/{file}')
        processed_image = image
        data.append(processed_image)
        labels.append(folder)


# Assume image_list is your list of images and label_list is your list of labels
# Convert the list of images to a NumPy array
X = np.array(data)/255.0

# Convert the list of labels to a NumPy array
labels = [list(map(int, label)) for label in labels]
y = np.array(labels)


# Assume X is your feature matrix and y is your multi-label target matrix
k_fold = IterativeStratification(n_splits=5, order=1)

def partial_correct_accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, 'bool')
    y_pred = tf.cast(tf.round(y_pred), 'bool')
    p = tf.reduce_sum(tf.cast(tf.logical_and(y_true, y_pred), 'float32'), axis=-1)
    q = tf.reduce_sum(tf.cast(tf.logical_or(y_true, y_pred), 'float32'), axis=-1)
    return tf.reduce_mean(tf.where(tf.equal(q, 0), 0.0, p / q))

def history_plot(train_history, i):
    # 繪製訓練和驗證的準確度、損失值和partial_correct_accuracy
    plt.figure(figsize=(18, 4))

    plt.subplot(1, 3, 1)
    plt.plot(train_history.history['accuracy'], label='Train Accuracy')
    plt.plot(train_history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Fold {i+1} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(train_history.history['loss'], label='Train Loss')
    plt.plot(train_history.history['val_loss'], label='Validation Loss')
    plt.title(f'Fold {i+1} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(train_history.history['partial_correct_accuracy'], label='Train Partial Correct Accuracy')
    plt.plot(train_history.history['val_partial_correct_accuracy'], label='Validation Partial Correct Accuracy')
    plt.title(f'Fold {i+1} Partial Correct Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Partial Correct Accuracy')
    plt.legend()

    plt.savefig(f'/content/fold_{i+1}_history.png')
    plt.close()

# 創建一個ImageDataGenerator對象來進行資料擴增
datagen = ImageDataGenerator(
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # rotation_range=30,
    # fill_mode='nearest',
    horizontal_flip=True,
)



# 在迴圈外部初始化一個列表來保存每次迭代的評估分數
scores = []
history_list = []
for i, (train_index, test_index) in enumerate(k_fold.split(X, y)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 48, 3)))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.6))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(5, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy', partial_correct_accuracy])



    # 使用ImageDataGenerator對象來擴增訓練數據
    train_gen = datagen.flow(X_train, y_train, batch_size=32)

    # 從訓練數據生成器中獲取一批數據
    data_batch, labels_batch = next(train_gen)

    # 繪製第一個圖片
    plt.figure(figsize=(6, 6))
    plt.imshow((data_batch[0]*255).astype('int'))
    plt.title('Augmented Image')
    # 儲存圖片
    plt.savefig('/content/augmented_image.png')
    plt.close()

    train_history = model.fit(
        X_train,
        y_train,
        epochs=80,
        validation_data=(X_test, y_test),
        verbose=0
    )

     # 在每次迭代結束後，評估模型並保存評估分數和訓練歷程
    score = model.evaluate(X_test, y_test)
    scores.append(score)
    history_list.append(train_history)

    history_plot(train_history, i)

# 計算平均分數
average_score = np.mean(scores, axis=0)
print(f'Average loss: {average_score[0]}')
print(f'Average accuracy: {average_score[1]}')
print(f'Average partial correct accuracy: {average_score[2]}')


In [ ]:
%cd /content/drive/MyDrive/mid/
from skmultilearn.model_selection import IterativeStratification
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.preprocessing.image import img_to_array, load_img
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from tensorflow.keras.utils import plot_model



folders = os.listdir('processed_data_all')

data = []
labels = []

for folder in tqdm(folders,desc='load image'):
    if not os.path.isdir(f'processed_data_all/{folder}'):
        continue
    for file in os.listdir(f'processed_data_all/{folder}'):
        image = cv2.imread(f'processed_data_all/{folder}/{file}')
        processed_image = image
        data.append(processed_image)
        labels.append(folder)


# Assume image_list is your list of images and label_list is your list of labels
# Convert the list of images to a NumPy array
X = np.array(data)/255.0

# Convert the list of labels to a NumPy array
labels = [list(map(int, label)) for label in labels]
y = np.array(labels)


# Assume X is your feature matrix and y is your multi-label target matrix
k_fold = IterativeStratification(n_splits=5, order=1)

def partial_correct_accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, 'bool')
    y_pred = tf.cast(tf.round(y_pred), 'bool')
    p = tf.reduce_sum(tf.cast(tf.logical_and(y_true, y_pred), 'float32'), axis=-1)
    q = tf.reduce_sum(tf.cast(tf.logical_or(y_true, y_pred), 'float32'), axis=-1)
    return tf.reduce_mean(tf.where(tf.equal(q, 0), 0.0, p / q))

def history_plot(train_history, i):
    # 繪製訓練和驗證的準確度、損失值和partial_correct_accuracy
    plt.figure(figsize=(18, 4))

    plt.subplot(1, 3, 1)
    plt.plot(train_history.history['accuracy'], label='Train Accuracy')
    plt.plot(train_history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Fold {i+1} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(train_history.history['loss'], label='Train Loss')
    plt.plot(train_history.history['val_loss'], label='Validation Loss')
    plt.title(f'Fold {i+1} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(train_history.history['partial_correct_accuracy'], label='Train Partial Correct Accuracy')
    plt.plot(train_history.history['val_partial_correct_accuracy'], label='Validation Partial Correct Accuracy')
    plt.title(f'Fold {i+1} Partial Correct Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Partial Correct Accuracy')
    plt.legend()

    plt.savefig(f'/content/fold_{i+1}_history.png')
    plt.close()

# 創建一個ImageDataGenerator對象來進行資料擴增
datagen = ImageDataGenerator(
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    # rotation_range=30,
    # fill_mode='nearest',
    horizontal_flip=True,
)



# 在迴圈外部初始化一個列表來保存每次迭代的評估分數
scores = []
history_list = []
for i, (train_index, test_index) in enumerate(k_fold.split(X, y)):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 48, 3)))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.6))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(5, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy', partial_correct_accuracy])



    # 使用ImageDataGenerator對象來擴增訓練數據
    train_gen = datagen.flow(X_train, y_train, batch_size=32)

    # 從訓練數據生成器中獲取一批數據
    data_batch, labels_batch = next(train_gen)

    # 繪製第一個圖片
    plt.figure(figsize=(6, 6))
    plt.imshow((data_batch[0]*255).astype('int'))
    plt.title('Augmented Image')
    # 儲存圖片
    plt.savefig('/content/augmented_image.png')
    plt.close()

    model.summary()
    plot_model(model, to_file='/content/model.png', show_shapes=True)

    train_history = model.fit(
        X_train,
        y_train,
        epochs=80,
        validation_data=(X_test, y_test),
        verbose=0
    )

     # 在每次迭代結束後，評估模型並保存評估分數和訓練歷程
    score = model.evaluate(X_test, y_test)
    scores.append(score)
    history_list.append(train_history)

    history_plot(train_history, i)

# 計算平均分數
average_score = np.mean(scores, axis=0)
print(f'Average loss: {average_score[0]}')
print(f'Average accuracy: {average_score[1]}')
print(f'Average partial correct accuracy: {average_score[2]}')


/content/drive/MyDrive/mid


load image: 100%|██████████| 32/32 [00:03<00:00,  8.86it/s]


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 62, 46, 32)        896       
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 31, 23, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_24 (Dropout)        (None, 31, 23, 32)        0         
                                                                 
 conv2d_17 (Conv2D)          (None, 29, 21, 64)        18496     
                                                                 
 batch_normalization_16 (Ba  (None, 29, 21, 64)        256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 14, 10, 64)       

KeyboardInterrupt: 

In [ ]:
%cd /content/drive/MyDrive/colab/hand_posture

import os
import cv2
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout
from keras import backend as K
def pipeline_process(image):
    image = resize(image, 64)
    ''' image = clear_pixels_by_hue(image)
    image = remove_noise(image) '''
    return image
# Step 1: 遍歷 raw_data 資料夾中的所有子資料夾
folders = os.listdir('processed_data')

data = []
labels = []

for folder in tqdm(folders,desc='load image'):
    if not os.path.isdir(f'processed_data/{folder}'):
        continue
    for file in os.listdir(f'processed_data/{folder}'):
        image = cv2.imread(f'processed_data/{folder}/{file}')
        processed_image = pipeline_process(image)
        data.append(processed_image)
        labels.append(folder)
        # 位移後的圖片
        M = np.float32([[1, 0, 10], [0, 1, 5]])  # 位移矩陣
        shifted_image = cv2.warpAffine(processed_image, M, (processed_image.shape[1], processed_image.shape[0]))
        data.append(shifted_image)
        labels.append(folder)
        M = np.float32([[1, 0, -10], [0, 1, -5]])  # 位移矩陣
        shifted_image2 = cv2.warpAffine(processed_image, M, (processed_image.shape[1], processed_image.shape[0]))
        data.append(shifted_image2)
        labels.append(folder)

# Step 3: 將處理後的圖片和對應的標籤（子資料夾名稱）存入一個數據集中
data = np.array(data)
labels = np.array(labels)
''' flipped_data = data[:, :, ::-1, :]
combined_data = np.concatenate((data, flipped_data), axis=0)
combined_labels = np.concatenate((labels, labels), axis=0) '''
# 將標籤二值化
lb = LabelBinarizer()
labels = [[int(bit) for bit in label] for label in labels]
# 將標籤二值化
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(labels)



/content/drive/.shortcut-targets-by-id/11gva2C9vGefcaGXnVGvkqhvm-5FkuR7i/colab/hand_posture


load image: 100%|██████████| 32/32 [01:51<00:00,  3.50s/it]

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 ...
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]


In [ ]:
def partial_correct_accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, 'bool')
    y_pred = tf.cast(tf.round(y_pred), 'bool')
    p = tf.reduce_sum(tf.cast(tf.logical_and(y_true, y_pred), 'float32'), axis=-1)
    q = tf.reduce_sum(tf.cast(tf.logical_or(y_true, y_pred), 'float32'), axis=-1)
    return tf.reduce_mean(tf.where(tf.equal(q, 0), 0.0, p / q))
# Step 4: 使用這個數據集來訓練一個模型，進行手指預測
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 48, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', partial_correct_accuracy])

train_history = model.fit(data, labels, epochs=10, validation_split = 0.1)

Epoch 1/10
27/27 [==============================] - 2s 17ms/step - loss: 5.1906 - accuracy: 0.2799 - partial_correct_accuracy: 0.4200 - val_loss: 0.6554 - val_accuracy: 0.4583 - val_partial_correct_accuracy: 0.6375
Epoch 2/10
27/27 [==============================] - 0s 8ms/step - loss: 0.6529 - accuracy: 0.3217 - partial_correct_accuracy: 0.4978 - val_loss: 0.5517 - val_accuracy: 0.4896 - val_partial_correct_accuracy: 0.8021
Epoch 3/10
27/27 [==============================] - 0s 8ms/step - loss: 0.5789 - accuracy: 0.3415 - partial_correct_accuracy: 0.5738 - val_loss: 0.4120 - val_accuracy: 0.3542 - val_partial_correct_accuracy: 0.8813
Epoch 4/10
27/27 [==============================] - 0s 8ms/step - loss: 0.4791 - accuracy: 0.3728 - partial_correct_accuracy: 0.6462 - val_loss: 0.4122 - val_accuracy: 0.3438 - val_partial_correct_accuracy: 0.8146
Epoch 5/10
27/27 [==============================] - 0s 9ms/step - loss: 0.3733 - accuracy: 0.3879 - partial_correct_accuracy: 0.7211 - val_loss

In [ ]:
from typing import List
import numpy as np


def partial_correct_accuracy(
    y_true,
    y_pred,
) -> float:

    count = 0

    for i in range(len(y_true)):
        p = sum(np.logical_and(y_true[i], y_pred[i]))
        q = sum(np.logical_or(y_true[i], y_pred[i]))

        count += p / q

    return count / len(y_true)
y_pred = model.predict(testX)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print("accuracy: ", partial_correct_accuracy(testY, y_pred))

2/2 [==============================] - 0s 8ms/step
accuracy:  0.5609375000000001
